In [ ]:
import os
import datetime
import ctypes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches

%matplotlib inline

In [ ]:
from IWFM_DLL import IWFM_Model

In [ ]:
dll_file = r'C:\Users\hatch\Desktop\IWFM\IWFM_DLL\IWFM2015_C_x64.dll'
#dll_file = r'C:\Users\hatch\Desktop\IWFM\IWFM_DLL\IWFM2015_C_x64_D.dll'

In [ ]:
preprocessor_file_name = r'C:\temp\C2VSimFG_v0.3.12\Preprocessor\C2VSimFG_Preprocessor.in'
simulation_file_name = r'C:\temp\C2VSimFG_v0.3.12\Simulation\C2VSimFG.in'

In [ ]:
x_coord_m = 564412
y_coord_m = 4491740
xy_fact = 3.2808

In [ ]:
c2vsimfg = IWFM_Model(dll_file, preprocessor_file_name, simulation_file_name)

In [ ]:
dir(c2vsimfg)

In [ ]:
c2vsimfg.get_subregions_by_element()

In [ ]:
c2vsimfg.get_element_info()

In [ ]:
%%time
begin_date = '09/30/1974_24:00'
end_date = '09/30/1975_24:00'

iLayer = ctypes.c_int(1)
cOutputBeginDateAndTime = ctypes.create_string_buffer(begin_date.encode('utf-8'))
cOutputEndDateAndTime = ctypes.create_string_buffer(end_date.encode('utf-8'))
iLenDateAndTime = ctypes.c_int(len(begin_date))
rFact_LT = ctypes.c_double(1.0)
iNNodes = ctypes.c_int(30179)
iNTime = ctypes.c_int(13)
rOutputDates = (ctypes.c_double*iNTime.value)()
rGWHeads = ((ctypes.c_double*iNNodes.value)*iNTime.value)()
#rGWHeads = ((ctypes.c_double*iNTime.value)*iNNodes.value)()
iStat = ctypes.c_int(-1)

c2vsimfg.dll.IW_Model_GetModelData_GWHeadsAll_ForALayer(ctypes.byref(iLayer),
                                 cOutputBeginDateAndTime,
                                 cOutputEndDateAndTime,
                                 ctypes.byref(iLenDateAndTime),
                                 ctypes.byref(rFact_LT),
                                 ctypes.byref(iNNodes),
                                 ctypes.byref(iNTime),
                                 rOutputDates,
                                 rGWHeads,
                                 ctypes.byref(iStat))

In [ ]:
heads = np.array(rGWHeads)

In [ ]:
heads[0]

In [ ]:
with open(r'C:\Users\hatch\Desktop\IWFM\IWFM_DLL\procedures_check.txt', 'w') as out_file:
    with open(r'C:\Users\hatch\Desktop\IWFM\IWFM_DLL\dll_procedures.txt', 'r') as in_file:
        for line in in_file:
            if hasattr(c2vsimfg.dll, line.strip()):
                check_procedure = 'PASS'
            else:
                check_procedure = 'FAIL'
                
            out_file.write("{} - {}\n".format(check_procedure, line.strip()))        

In [ ]:
c2vsimfg.get_element_config(1)

In [ ]:
c2vsimfg.get_node_ids()

In [ ]:
c2vsimfg.get_node_coordinates()

In [ ]:
c2vsimfg.get_element_info()

In [ ]:
elem_data = c2vsimfg.get_element_spatial_info()

In [ ]:
fig, ax =plt.subplots(figsize=(6,11))
c2vsimfg.plot_elements(ax) #, write_to_file=True, file_name=r'C:\temp\C2VSimFG_v0.3.5\elements.png')

In [ ]:
version_info = c2vsimfg.get_version()
version_info.split('\n')

In [ ]:
c2vsimfg.get_n_layers()

In [ ]:
c2vsimfg.get_stratigraphy_atXYcoordinate(x_coord_m, y_coord_m, xy_fact, output_options='combined')

In [ ]:
c2vsimfg.get_location_type_id('gw_head_obs')

In [ ]:
c2vsimfg.get_hydrograph_coordinates('gw_head_obs')

In [ ]:
c2vsimfg.get_names('gw_head_obs')

In [ ]:
data_list = c2vsimfg.get_data_list('element')
if data_list:
    print(data_list)

In [ ]:
c2vsimfg.get_n_hydrographs('gw_head_obs')

In [ ]:
c2vsimfg.get_n_hydrographs('tile_drain')

In [ ]:
c2vsimfg.get_n_hydrographs('stream_hydrograph_obs')

In [ ]:
c2vsimfg.get_n_hydrographs('subsidence_obs')

In [ ]:
c2vsimfg.get_n_hydrographs('node')

In [ ]:
start_date, end_date, time_interval = c2vsimfg.get_time_specs()

In [ ]:
print(start_date)
print(end_date)
print(time_interval)

In [ ]:
c2vsimfg.get_n_data_list('node')

In [ ]:
c2vsimfg.get_data_list('node')

In [ ]:
c2vsimfg.get_sub_data_list('node', 0)

In [ ]:
c2vsimfg._has_sub_data_list('tile_drain', 0)

In [ ]:
c2vsimfg.get_output_interval('subregion', 0)

In [ ]:
c2vsimfg.get_n_intervals(start_date, end_date, time_interval)

In [ ]:
c2vsimfg.is_date_greater(start_date, end_date)

In [ ]:
c2vsimfg.get_data_unit_id('volume')

In [ ]:
#dts, data = c2vsimfg.get_model_data('gw_head_obs', 1, 0, -1, start_date, end_date, time_interval)
dts, data = c2vsimfg.get_model_data('node', 1, 0, -1, start_date, end_date, time_interval)

In [ ]:
data

In [ ]:
hydrograph_stratigraphy = c2vsimfg.get_hydrograph_info('gw_head_obs')

In [ ]:
hydrograph_stratigraphy

In [ ]:
dates, heads = c2vsimfg.get_gwheadsall_foralayer(1, '09/30/2010', '09/30/2015')

In [ ]:
count_well_names = hydrograph_stratigraphy.groupby('Name')['id'].count().to_frame()
count_well_names.reset_index(inplace=True)

In [ ]:
count_well_names[count_well_names['id'] > 4]

In [ ]:
hydrograph_id = 1
hydrograph_df = pd.DataFrame({'date': dts, 'head': data})
hydrograph_df['GSE'] = hydrograph_stratigraphy[hydrograph_stratigraphy['id']==hydrograph_id]['GSE'].values[0]

In [ ]:
observed_data_path = r'C:\temp\C2VSimFG_v0.3.5\ObservedData'
observed_data = 'Observed_GW_Levels.csv'
observed_gw_levels = pd.read_csv(os.path.join(observed_data_path, observed_data))
observed_gw_levels['Date'] = pd.to_datetime(observed_gw_levels['Date'], format='%Y-%m-%d')

In [ ]:
hydrograph_stratigraphy[hydrograph_stratigraphy['Name'].isin(observed_gw_levels['Well_Name'].unique())]

In [ ]:
observed_gw_levels.info()

In [ ]:
def closest_end_of_month(dt):
    ''' returns the date for the end of the closest month to the date provided
    
    Parameters
    ----------
    dt : datetime.datetime object
        date being converted to end of closest month
        
    Returns
    -------
    datetime.datetime
        date for the end of the month closest to the date provided
    '''
    # get number of days between date and end of the previous month
    previous_delta = datetime.timedelta(days=dt.day)
    
    # determine end of the current month
    next_month = dt.replace(day=28) + datetime.timedelta(days=4)
    end_of_current_month = next_month - datetime.timedelta(days=next_month.day)
    
    # get number of days between date and end of the current month
    next_delta = end_of_current_month - dt
    
    # use the smaller delta to return the closest
    if previous_delta < next_delta:
        return dt - previous_delta
    else:
        return end_of_current_month

In [ ]:
observed_gw_levels['Model_Date'] = observed_gw_levels['Date'].apply(closest_end_of_month)

In [ ]:
# set path for saving plots
fig_path = r'C:\temp\C2VSimFG_v0.3.5\Results\Plots'

# generate figure geometry
fig = plt.figure(figsize=(17,11))
gs = fig.add_gridspec(ncols=7, nrows=6)
ax1 = fig.add_subplot(gs[:,:3])
ax2 = fig.add_subplot(gs[0:2,3:])
ax3 = fig.add_subplot(gs[2:4,3:5])
ax4 = fig.add_subplot(gs[2:4,5:])
ax5 = fig.add_subplot(gs[4:,3:5])
ax6 = fig.add_subplot(gs[4:,5:])
ax = (ax3, ax4, ax5, ax6)

for well_name in observed_gw_levels['Well_Name'].unique():
    hydrograph_ids = hydrograph_stratigraphy[hydrograph_stratigraphy['Name'] == well_name]['id']
      
    # skip locations where number of hydrographs with a given name exceeds the number of model layers (i.e. duplicates)
    if len(hydrograph_ids) > c2vsimfg.get_n_layers():
        continue
    
    for i, hydrograph_id in enumerate(hydrograph_ids):
        
        # get simulated hydrograph time series
        dts, data = c2vsimfg.get_model_data('gw_head_obs', hydrograph_id, 0, -1, start_date, end_date, time_interval)
        hydrograph_df = pd.DataFrame({'date': dts, 'head': data})
        
        # add column for ground surface elevation to simulated hydrograph for plotting
        hydrograph_df['GSE'] = hydrograph_stratigraphy[hydrograph_stratigraphy['id']==hydrograph_id]['GSE'].values[0]
        
        # create simulated v. observed comparision dataset
        simulated_v_observed = observed_gw_levels[observed_gw_levels['Well_Name']==well_name].merge(hydrograph_df, left_on='Model_Date',right_on='date')
        label = 'simulated layer {}'.format(i+1)
        
        # plot model elements
        c2vsimfg.plot_elements(ax1)
        
        # plot all observation well locations
        ax1.plot(hydrograph_stratigraphy['X'], hydrograph_stratigraphy['Y'], 'k.', markersize=4)
        
        # plot observation well of interest
        ax1.plot(hydrograph_stratigraphy[hydrograph_stratigraphy['Name'] == well_name]['X'], 
                 hydrograph_stratigraphy[hydrograph_stratigraphy['Name'] == well_name]['Y'], 'ro')
        
        # annotate observation well of interest
        ax1.annotate(well_name, 
                     xy=(hydrograph_stratigraphy[hydrograph_stratigraphy['Name'] == well_name]['X'].values[0],
                         hydrograph_stratigraphy[hydrograph_stratigraphy['Name'] == well_name]['Y'].values[0]),
                     xytext=(hydrograph_stratigraphy[hydrograph_stratigraphy['Name'] == well_name]['X'].values[0] + 100000, 
                             hydrograph_stratigraphy[hydrograph_stratigraphy['Name'] == well_name]['Y'].values[0] - 50000), 
                     arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=0.3", relpos=(0., 0.)))
        
        # plot simulated hydrograph
        ax2.plot('date', 'head', data=hydrograph_df, label=label)
        
        # plot simulated v. observed scatter plots
        ax[i].plot('GWL', 'head', 'o', data=simulated_v_observed, markerfacecolor='none')
        ax[i].plot('GWL', 'GWL', 'k', data=observed_gw_levels[observed_gw_levels['Well_Name']==well_name])
        ax[i].set_title("Simulated v. Observed for Layer {}".format(i+1))
        ax[i].set_xlabel('Observed Head (ft)')
        ax[i].set_ylabel('Simulated Head (ft)')
        ax[i].grid()
    
    # plot observed data hydrograph
    ax2.plot('Date', 'GWL', 'o', data=observed_gw_levels[observed_gw_levels['Well_Name'] == well_name], 
             markerfacecolor='none', label='observed')
    
    # plot ground surface elevation
    ax2.axhline(hydrograph_stratigraphy[hydrograph_stratigraphy['id']==hydrograph_id]['GSE'].values[0],
                color='brown',
                label='ground surface')
    
    # format plot layout
    ax1.grid()
    ax2.set_title('Groundwater Hydrographs for Well {}'.format(well_name))
    ax2.set_ylabel('Head (ft)')
    ax2.set_xlabel('Time')
    ax2.set_xlim(left=hydrograph_df['date'].min(), right=hydrograph_df['date'].max())
    ax2.grid()
    ax2.legend(loc='right', ncol=1, bbox_to_anchor=(1.0, 0.75))
    gs.tight_layout(fig)
    
    # save figure to file
    fig_name = 'Hydrograph_Well_{}.png'.format(well_name)
    plt.savefig(os.path.join(fig_path, fig_name))
    
    # clear axes
    ax1.cla()
    ax2.cla()
    ax3.cla()
    ax4.cla()
    ax5.cla()
    ax6.cla()
    
# clear figure
plt.clf()
plt.close('all')

In [ ]:
fig = plt.figure(figsize=(17,11))
gs = fig.add_gridspec(ncols=7, nrows=6)
ax1 = fig.add_subplot(gs[:,:3])
ax2 = fig.add_subplot(gs[0:2,3:])
ax3 = fig.add_subplot(gs[2:4,3:5])
ax4 = fig.add_subplot(gs[2:4,5:])
ax5 = fig.add_subplot(gs[4:,3:5])
ax6 = fig.add_subplot(gs[4:,5:])
# plot model elements
c2vsimfg.plot_elements(ax1)

# get plot bounds for map
xmin, xmax = ax1.get_xlim()
ymin, ymax = ax1.get_ylim()

print(xmin)
print(xmax)
print(ymin)
print(ymax)

# plot all observation well locations
ax1.plot(hydrograph_stratigraphy['X'], hydrograph_stratigraphy['Y'], 'k.', markersize=4)

well_name = observed_gw_levels['Well_Name'].unique()[0]

# plot observation well
ax1.plot(hydrograph_stratigraphy[hydrograph_stratigraphy['Name'] == well_name]['X'], 
         hydrograph_stratigraphy[hydrograph_stratigraphy['Name'] == well_name]['Y'], 'ro')

# annotate observation well
ann = ax1.annotate('Observation Well:\n{}'.format(well_name), 
             xy=(hydrograph_stratigraphy[hydrograph_stratigraphy['Name'] == well_name]['X'].values[0], hydrograph_stratigraphy[hydrograph_stratigraphy['Name'] == well_name]['Y'].values[0]),
             xytext=(hydrograph_stratigraphy[hydrograph_stratigraphy['Name'] == well_name]['X'].values[0] + 100000, hydrograph_stratigraphy[hydrograph_stratigraphy['Name'] == well_name]['Y'].values[0] - 50000), 
             arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=0.3", relpos=(0., 0.)))

# need to get bounding box for annotation text in data coordinates
# need to get closest max and min on x-axis for model elements to y-coordinate of annotation in data coordinates
# default label is to right and down of figure with arrow from lower left of text
# check that right side of text label does not exceed axis limits
# check that left side of text label is greater than the max of the closest model elements



# set axis titles
ax3.set_title("Simulated v. Observed for Layer {}".format(i+1))
ax3.set_xlabel('Observed Head (ft)')
ax3.set_ylabel('Simulated Head (ft)')

# set figure title
#fig.suptitle('This is a somewhat long figure title', fontsize=16)

# adjust bounds to fit labels
gs.tight_layout(fig)

In [ ]:
data

In [ ]:
df

In [ ]:
def is_valid_feature_type(feature_type):
    ''' checks that a provided feature type is a valid iwfm feature
    
    Parameters
    ----------
    feature_type : str
        value to be checked to see if it is a valid IWFM feature type and can be used to generate
        a location type id
        
    Returns
    -------
    boolean
        True if a valid IWFM feature type
        i.e. node, element, subregion, zone, lake, stream_node, stream_reach, tile_drain,
        small_watershed, gw_head_obs, stream_hydrograph_obs, or subsidence_obs
        False otherwise
    '''
    # check input type is a string
    if not isinstance(feature_type, str):
        raise TypeError('feature_type must be a string. type entered is {}.'.format(type(feature_type)))
    
    # list of valid feature types
    _valid_feature_types = ['node',
                            'element',
                            'subregion', 
                            'zone',
                            'lake',
                            'stream_node',
                            'stream_reach',
                            'tile_drain',
                            'small_watershed',
                            'gw_head_obs',
                            'stream_hydrograph_obs',
                            'subsidence_obs'
                            ]
    
    if feature_type.lower() not in _valid_feature_types:
        return False
    
    return True   

In [ ]:
is_valid_feature_type('')

In [ ]:
def validate_feature_type(feature_type):
    ''' performs validation that a provided feature_type is a valid IWFM feature type
    
    Parameters
    ----------
    feature_type : str
        value to be validated that it can be used to generate a location type id
        
    Returns
    -------
    None
        raises errors if validation checks do not pass
    '''
    # check input type is a string
    if not isinstance(feature_type, str):
        raise TypeError('feature_type must be a string. type entered is {}.'.format(type(feature_type)))
    
    # list of valid feature types
    _valid_feature_types = ['node',
                            'element',
                            'subregion', 
                            'zone',
                            'lake',
                            'stream_node',
                            'stream_reach',
                            'tile_drain',
                            'small_watershed',
                            'gw_head_obs',
                            'stream_hydrograph_obs',
                            'subsidence_obs'
                            ]
    
    if feature_type.lower() not in _valid_feature_types:
        error_string = 'feature_type entered is not a valid IWFM feature type.\n' + \
                       'feature type must be:\n\t-{}'.format('\n\t-'.join(_valid_feature_types))
        raise ValueError(error_string)

In [ ]:
validate_feature_type('node')

In [ ]:
def _is_valid_iwfm_date(dt):
    ''' returns True if date is IWFM-format date string based on:
        string length and format MM/DD/YYYY_HH:MM

    Parameters
    ----------
    dt : str
        input value to check if IWFM-format date

    Returns
    -------
    boolean
        True if format is IWFM date
        False if not an IWFM date 
    '''
    if not isinstance(dt, str):
        raise TypeError('IWFM dates must be a string')
                
    if len(dt) != 16:
        return False
            
    if '_' not in dt or dt.index('_') != 10:
        return False
            
    if ':' not in dt or dt.index(':') != 13:
        return False
            
    if dt[2] != '/' or dt[5] != '/':
        return False
            
    try:
        datetime.datetime.strptime(dt, '%m/%d/%Y_%H:%M')
    except ValueError:
        try:
            datetime.datetime.strptime(dt.split('_')[0], '%m/%d/%Y')
        except ValueError:
            return False
        else:
            try:
                hour = int(dt[11:13])
                minute = int(dt[15:])
            except:
                return False
            else:
                if hour < 0 or hour > 24:
                    return False
                        
                if minute < 0 or minute > 59:
                    return False
                        
                return True
    else:
        return True

In [ ]:
def validate_iwfm_date(dt):
    ''' performs validation that a provided value is an IWFM-format date string based on
    string length and format MM/DD/YYYY_HH:MM

    Parameters
    ----------
    dt : str
        input value to check if IWFM-format date

    Returns
    -------
    None
        raises errors if validation checks do not pass
    '''
    if not isinstance(dt, str):
        raise TypeError('IWFM dates must be a string')
                
    if len(dt) != 16:
        raise ValueError("IWFM dates must be 16 characters in length and of format MM/DD/YYYY_HH:MM")
            
    if '_' not in dt or dt.index('_') != 10:
        raise ValueError("IWFM dates must have an '_' separating the date and time")
            
    if ':' not in dt or dt.index(':') != 13:
        raise ValueError("IWFM dates must have an ':' separating the hours from minutes")
            
    if dt[2] != '/' or dt[5] != '/':
        raise ValueError("IWFM dates must use '/' as separators for the month, day, year in the date")
            
    try:
        datetime.datetime.strptime(dt, '%m/%d/%Y_%H:%M')
    except ValueError:
        try:
            datetime.datetime.strptime(dt.split('_')[0], '%m/%d/%Y')
        except ValueError:
            raise ValueError("Value provided: {} could not be converted to a date".format(dt))
        else:
            try:
                hour = int(dt[11:13])
                minute = int(dt[14:])
            except:
                raise ValueError("hour or minute values are not numbers")
            else:
                if hour < 0 or hour > 24:
                    raise ValueError("hour value is not between 00 and 24")
                        
                if minute < 0 or minute > 59:
                    raise ValueError("minute value is not between 00 and 59")
                    
                if hour == 24 and minute > 0:
                    raise ValueError("days cannot exceed 24:00 hours")

In [ ]:
d = '02/28/2015_24:00'
validate_iwfm_date(d)

In [ ]:
def _validate_time_interval(time_interval):
    ''' performs validation that a provided value is an IWFM-format time-interval string

    Parameters
    ----------
    time_interval : str (not case-sensitive)
        input value to check if IWFM-format time-interval

    Returns
    -------
    None
        raises errors if validation checks do not pass
    '''
    # check input type is a string
    if not isinstance(time_interval, str):
        raise TypeError('time_interval must be a string. type entered is {}.'.format(type(time_interval)))
        
    # list of valid time intervals
    _valid_time_intervals = ['1MIN',
                             '2MIN',
                             '3MIN',
                             '4MIN',
                             '5MIN',
                             '10MIN',
                             '15MIN',
                             '20MIN',
                             '30MIN',
                             '1HOUR',
                             '2HOUR',
                             '3HOUR',
                             '4HOUR',
                             '6HOUR',
                             '8HOUR',
                             '12HOUR',
                             '1DAY',
                             '1WEEK',
                             '1MON',
                             '1YEAR'
                            ]
    
    if time_interval.upper() not in _valid_time_intervals:
        error_string = 'time_interval entered is not a valid IWFM time interval.\n' + \
                       'time_interval must be:\n\t-{}'.format('\n\t-'.join(_valid_time_intervals))
        raise ValueError(error_string)    

In [ ]:
s = '1mon'

In [ ]:
validate_time_interval(s)